In [3]:
import time
import copy

In [4]:
class ttt_plate():
    def __init__(self):
        self.reset()     
    
    def draw(self,pos,symbol):
        # Draw a single symbol on the plate
        # param pos: position of target cell, (int,int)
        # param symbol: symbol to draw, str
        if symbol in ("O", "X"):
            if not self.plate[pos[0]][pos[1]] == " ":
                print(pos)
                self.print_plate()               
                assert self.plate[pos[0]][pos[1]] == " "
            self.plate[pos[0]][pos[1]] = symbol
        else:
            raise ValueError()
    def judge(self):
        winner = " "
        state = self.plate
        for i in range(3):
            if state[i][0] == state[i][1] and state[i][1] == state[i][2] and state[i][2] != " ":
                winner = state[i][0]
            elif state[0][i] == state[1][i] and state[1][i] == state[2][i] and state[2][i] != " ":
                winner =state[0][i]
        if state[0][0] == state[1][1] and state[1][1] == state[2][2] and state[2][2] != " ":
            winner = state[0][0]
        elif state[2][0] == state[1][1] and state[1][1] == state[0][2] and state[2][0] != " ":
            winner = state[1][1]
        return winner  
    def reset(self):
        self.plate = [[" ", " ", " "] for _ in range(3)]
        
    def print_plate(self):
        for i in range(3):
            print(self.plate[i])
    
    def count_empty(self):
        count = 0
        for i in range(9):
            if self.plate[i // 3][i % 3] == " ":
                count += 1
        return count

In [7]:
class ttt_agent():
    def __init__(self, symbol):
        self.symbol = symbol
        self.opponent = "O" if symbol == "X" else "X"
        
    def action(self, ttt_plate, symbol=None):
        # 입력해야 하는 Symbol을 등록
        if symbol is None:
            symbol = self.symbol
        
        # Agent Simulation에 사용할 tic-tac-toe Plate를 복사
        copied_ttt_state = copy.deepcopy(ttt_plate)
        
        action = (-1,-1) # placeholder         
        
        # 탐색하지 않은 Search Tree의 각 Node를 저장
        tree_queue = [(copied_ttt_state,
                       action)] # (현재 state, Root Node에서 처음 선택한 Action)
        score_of_action = [0 for i in range(9)]
        while len(tree_queue) > 0:
            cur_state, init_action = tree_queue.pop(0) # (현재 state, Root Node에서 처음 선택한 Action)
            # 자식 노드 중 탐색하지 않은 노드가 있는지 여부
            empty_exists = False
            
            # 모든 Cell에 대해서,
            for s in range(9):
                x = s // 3
                y = s % 3
                # 빈 Cell이 있으면 (=내 차례에 뭘 칠할게 있다면)
                if cur_state.plate[x][y] == " ":
                    
                    # Agent Simulation을 위한 깊은 복사
                    temp_ttt = copy.deepcopy(cur_state)
                    # 해당 Cell에 현재 차례에 해당하는 Symbol을 그린다.
                    temp_ttt.draw((x,y),symbol)
                    
                    # 이겼는지 확인 (의미 없는 추가 탐색을 막기 위함)
                    winner = temp_ttt.judge()
                    if winner == symbol: 
                        score_of_action[s] += 1
                    else:
                        # 더 그릴 Cell이 남아 있으면
                        if temp_ttt.count_empty() > 0:
                            # 적 (Min agent)의 행동을 에측한다
                            opponent_symbol = "O" if symbol == "X" else "X"
                            opponent_action = self.action(temp_ttt,opponent_symbol)
                            
                            # Min Agent의 행동을 시뮬레이션에 반영한다.
                            temp_ttt.draw(opponent_action,opponent_symbol)
                            
                            # 적이 이겼는지 확인
                            winner = temp_ttt.judge()
                            if winner == opponent_symbol: 
                                score_of_action[s] -= 1
                            else:  
                                # Search Tree에서 탐색해야 하는 Cell을 Queue에 입력한다.
                                tree_queue.append((temp_ttt,
                                                   (x,y) if init_action == (-1,-1) else init_action))
                                empty_exists = True
            
            # 9칸을 모두 칠한 경우, 혹은 현재 State에서 승자가 결정된 경우
            winner = cur_state.judge()
            if not empty_exists or winner != " ":
                if winner == self.symbol:
                    score = 1
                elif winner == " ":
                    score = 0
                else:
                    score = -1
                # Root Node에서 처음 선택한 Action의 Benefit 값을 변경
                score_of_action[init_action[0] * 3 + init_action[1]] += score
        
        # 가능한 Action 중 Benefit 값이 큰 Action을 선택
        max_score = -100
        for idx,score in enumerate(score_of_action):
            x = idx // 3
            y = idx % 3
            if score > max_score and ttt_plate.plate[x][y] == " ":
                max_score = score
                action = (x,y)
            
        return action 

In [8]:
play = ttt_plate()
player_1 = ttt_agent("O")
player_2 = ttt_agent("X")

# If two players plays perfect, there must be no winner
for i in range(4):
    p1_act = player_1.action(play)
    play.draw(p1_act,"O")
    play.print_plate()
    p2_act = player_2.action(play)
    play.draw(p2_act,"X")
    play.print_plate()

p1_act = player_1.action(play)
play.draw(p1_act,"O")
# If two player plays perfect, there must be NO winner
assert play.judge() == " "
play.print_plate()

[' ', ' ', ' ']
[' ', 'O', ' ']
[' ', ' ', ' ']
[' ', ' ', 'X']
[' ', 'O', ' ']
[' ', ' ', ' ']
['O', ' ', 'X']
[' ', 'O', ' ']
[' ', ' ', ' ']
['O', ' ', 'X']
[' ', 'O', ' ']
[' ', ' ', 'X']
['O', ' ', 'X']
[' ', 'O', 'O']
[' ', ' ', 'X']
['O', ' ', 'X']
['X', 'O', 'O']
[' ', ' ', 'X']
['O', 'O', 'X']
['X', 'O', 'O']
[' ', ' ', 'X']
['O', 'O', 'X']
['X', 'O', 'O']
[' ', 'X', 'X']
['O', 'O', 'X']
['X', 'O', 'O']
['O', 'X', 'X']
